In [ ]:
def inversion_fit(G:torch.nn.DataParallel, # генеративная модель
                  D:torch.nn.DataParallel, # модель дискриминатор
                  T:torch.nn.DataParallel, # целевая модель 
                  iden:torch.Tensor, # начальные метки
                  lr=1e-2,
                  momentum=0.9,
                  lamda=100,
                  iter_times=1500,
                  clip_range=1):
	iden = iden.view(-1).long()
	criterion = nn.CrossEntropyLoss() # нужна для Total_Loss
	bs = iden.shape[0]
	
	G.eval() 
	D.eval()
	T.eval()

	max_score = torch.zeros(bs)
	max_iden = torch.zeros(bs)
	z_hat = torch.zeros(bs, 100)
	
	for random_seed in tqdm(range(5)):
		tf = time.time()
		
		torch.manual_seed(random_seed) 
		torch.cuda.manual_seed(random_seed) 
		np.random.seed(random_seed) 
		random.seed(random_seed)

		z = torch.randn(bs, 100).float() # начальный шум, необходимый генератору для производства картинок
		z.requires_grad = True
		v = torch.zeros(bs, 100).float()
			
		for i in range(iter_times):
			fake = G(z) # генерируем изображение
			label = D(fake # получаем метку класса от дискриминатора, обученного на открытом датасете
			out = T(fake)[-1] # получаем метку класса от целевой модели, обученной на закрытом датасете
			
			if z.grad is not None:
				z.grad.data.zero_()

			Prior_Loss = - label.mean()
			Iden_Loss = criterion(out, iden)
			Total_Loss = Prior_Loss + lamda * Iden_Loss # вычисляем ошибку как лин. комбинацию от ошибки дискриминатора и ошибки целевой модели

			Total_Loss.backward() # Back Propagation
			
			v_prev = v.clone()
			gradient = z.grad.data
			v = momentum * v - lr * gradient
			z = z + ( - momentum * v_prev + (1 + momentum) * v) # Ищем оптимальное зачение начального шума, опираясь на гиперпараметры и пред. значение
			z = torch.clamp(z.detach(), -clip_range, clip_range).float()
			z.requires_grad = True
            
			Prior_Loss_val = Prior_Loss.item()
			Iden_Loss_val = Iden_Loss.item()
			if (i+1) % 300 == 0:
				fake_img = G(z.detach())
				eval_prob = T(fake_img)[-1]
				eval_iden = torch.argmax(eval_prob, dim=1).view(-1)
				acc = iden.eq(eval_iden.long()).sum().item() * 1.0 / bs		# метрики и промежуточные примеры
		fake = G(z)
		score = T(fake)[-1]
		eval_iden = torch.argmax(score, dim=1).view(-1)
        
	return G(z) # возвращаем результат работы генератора